In [1]:
from weasyprint import HTML, CSS
from weasyprint.text.fonts import FontConfiguration

import pandas as pd
from pathlib import Path

In [2]:
cities_df = pd.read_csv('Permits in 2018-2021 vs. 6th cycle RHNA.csv', index_col=0)

cities_df = cities_df[
    ~cities_df['Jurisdiction'].str.contains('Unincorporated')
    # Don't include cities that are already making sufficient progress.
    & (cities_df['fraction'] < 1)
]

In [3]:
cities_df[
    cities_df['Jurisdiction'].str.contains('Fr')
]

,Jurisdiction,6th cycle RHNA,County,2018,2019,2020,2021,Average rate,8-year rate,fraction
5,Fremont,12897,Alameda,1863.0,1087.0,311.0,860.0,1030.25,8242.0,0.64
53,San Francisco,82069,San Francisco,6097.0,3297.0,2168.0,NaN,3854.00,30832.0,0.38
72,South San Francisco,3956,San Mateo,167.0,295.0,502.0,148.0,278.00,2224.0,0.56


In [21]:
header = """
<div class="text-center">
    <img class="h-20 inline"
    src="file:///Users/sidharth.kapur/personal-workspace/yimby-emails/YIMBY Law logo.png" />
    <img class="h-20 inline"
    src="file:///Users/sidharth.kapur/personal-workspace/yimby-emails/Greenbelt Alliance logo.png" />
</div>
"""

body = """
April 21, 2022

Dear {city} City Council:

We are writing on behalf of <b class="text-bold">YIMBY Law</b> and 
<b class="text-bold">Greenbelt Alliance</b> regarding {city}’s 6th Cycle Housing 
Element Update. 
<b class="text-bold">YIMBY Law</b> is a legal nonprofit working to make housing 
in California more accessible and affordable through enforcement of state law. 
<b class="text-bold">Greenbelt Alliance</b> is an environmental nonprofit working 
to ensure that the Bay Area’s lands and communities are resilient to a changing climate.

We are writing to remind you of {city}'s obligation to include sufficient sites 
in your upcoming Housing Element to accommodate your Regional Housing Needs 
Allocation (RHNA) of <b class="text-bold">{rhna} units</b>. 

In the Annual Progress Reports that {city} submitted to HCD, we
observe the following trend of housing units permitted in the last {num_years} years:

<table class="mt-2 border mx-auto text-center text-sm">
<tr>
    <th {cell_classes}>Year</th>
    <th {cell_classes}>Housing units permitted</th>
</tr>
{table_body}
<tr>
    <td class="border p-1 font-bold">Average, 2018-{end_year}</td>
    <td class="border p-1 font-bold">{units_average}</td>
</tr>
</table>

To meet the 6th cycle RHNA target, the rate of new housing 
permits in {city} would need 
to increase from <b class="text-bold">{units_average} units per year</b> 
in 2018-{end_year} to <b class="text-bold">{required_yearly_rate} units per year</b> 
in the next 8 years. 
This is a {percent_increase} increase from recent years. 
<b class="text-bold">If the current pace were to continue, {city} would meet 
only {current_fraction} of its new housing target.</b>

Based on these trends, it is unlikely that {city}’s existing realistic zoning capacity 
is sufficient to meet its 6th cycle RHNA target. 
According to HCD’s 
<a href="https://www.hcd.ca.gov/community-development/housing-element/docs/sites_inventory_memo_final06102020.pdf">Housing Element Site Inventory Guidebook</a>, 
housing elements must analyze the realistic capacity of their sites, which may include 
considerations of “[l]ocal or regional track records”, “past production trends”, 
and “the rate at which similar parcels were developed during the previous planning period”.
A housing element that does not include a significant rezoning component is 
therefore unlikely to be compliant with state law.

We urge {city} to include a major rezoning component in its 
Housing Element—a rezoning large enough to close the gap between recent 
housing production trends and the RHNA target. The rezoning should be 
within existing communities and should comply with the city’s obligation to 
Affirmatively Further Fair Housing. We also urge {city} to ease any other constraints, 
such as discretionary approval processes or impact fees, that may impede the rate of
development on your city's housing sites.

<br>

Thank you,

<table>
<tr class="mb-1">
    <td><b class="text-bold">Sid Kapur</b>, East Bay YIMBY</td>
    <td class="px-1">(sidharthkapur1@gmail.com)</td>
</tr>
<tr class="mb-1">
    <td><b class="text-bold">Rafa Sonnenfeld</b>, YIMBY Law</td>
    <td class="px-1">(rafa@yimbylaw.org)</td>
</tr>
<tr>
    <td><b class="text-bold">Zoe Siegel</b>, Greenbelt Alliance</td>
    <td class="px-1">(zsiegel@greenbelt.org)</td>
</tr>
</table>
"""

row_template = """
    <tr>
        <td {cell_classes}>{year}</td>
        <td {cell_classes}>{value}</td>
    </tr>
"""

font_css = CSS(string="""
h3 {
    font-family: "Founders Grotesk";
}
h1, h2, p {
    font-family: ETbb;
}

p {
    hyphens: auto;
    font-size: 14px;
}

@page {
    margin-left: 2cm;
    margin-right: 2cm;
    margin-top: 1cm;
    margin-bottom: 1cm;
}
"""
)
font_config = FontConfiguration()

tailwind_css = CSS(url='https://unpkg.com/tailwindcss@^2/dist/tailwind.min.css')

In [22]:
# for _, row in cities_df.loc[lambda x: x['Jurisdiction'] == 'Alameda'].iterrows():
for _, row in cities_df.iterrows():
    city = row['Jurisdiction']
    rhna = row['6th cycle RHNA']
    
    units_2018 = row['2018']
    units_2019 = row['2019']
    units_2020 = row['2020']
    units_2021 = row['2021']
    end_year = 2021 if pd.notnull(units_2021) else 2020
    
    average_rate = row['Average rate']
    eight_year_rate = row['8-year rate']
    
    required_yearly_rate = rhna / 8
    
    current_fraction = eight_year_rate / rhna
    percent_increase = rhna / eight_year_rate - 1
    
    NUMBER_FORMAT = '{:,.0f}'
    PERCENT_FORMAT = '{:.0%}'
    
    table_body = "".join([
        row_template.format(
            cell_classes='{cell_classes}',
            year=year,
            value=NUMBER_FORMAT.format(value),
        )
        for year, value in [
            (2018, units_2018),
            (2019, units_2019),
            (2020, units_2020),
            (2021, units_2021),
        ]
        if pd.notnull(value)
    ])
    cell_classes = 'class="border p-1"'

    body_formatted = body.format(
        city=city,
        rhna=NUMBER_FORMAT.format(rhna),
        cell_classes=cell_classes,
        table_body=table_body.format(cell_classes=cell_classes),
        end_year=end_year,
        num_years='three' if end_year == 2020 else 'four',
        required_yearly_rate=NUMBER_FORMAT.format(required_yearly_rate),
        units_average=NUMBER_FORMAT.format(average_rate),
        percent_increase=PERCENT_FORMAT.format(percent_increase),
        eight_year_rate=NUMBER_FORMAT.format(eight_year_rate),
        current_fraction=PERCENT_FORMAT.format(current_fraction),
    )

    body_html = ''.join(
        [
            f'<p class="mb-3 text-justify">{para}</p>' 
            for para in body_formatted.split('\n\n')
        ]
    )
    html_str = f"""
    {header}
    <html lang="en">
    <main class="mt-4">
        {body_html}
    </main>
    </html>
    """

    html = HTML(string=html_str)

    html.write_pdf(f'./output/{city}.pdf', stylesheets=[tailwind_css, font_css], font_config=font_config)
    
    # dump a preview so I can share it on Google Docs for edits
    Path(f'./html/{city}.html').write_text(
        """
        <link rel="stylesheet" href="https://unpkg.com/tailwindcss@^2/dist/tailwind.min.css">
        <style>
        {font_css.string}
        </style>
        """
        + html_str
    )

1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
1 extra bytes in post.stringData array
'created' timestamp seems very low